In [17]:
import random

class Card:
    def __init__(self, suit, rank, value):
        self.suit = suit
        self.rank = rank
        self.value = value

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self):
        self.cards = []
        self.build()

    def build(self):
        suits = ["Hearts", "Diamonds", "Clubs", "Spades", "Stars", "Moons"]
        ranks = {
            "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7,
            "8": 8, "9": 9, "10": 10, "Jack": 11, "Queen": 12, "King": 13, "Ace": 14,
            "Wizard": 15, "Dragon": 16, "Phoenix": 17
        }
        for suit in suits:
            for rank, value in ranks.items():
                self.cards.append(Card(suit, rank, value))

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        return self.cards.pop() if self.cards else None

class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []
        self.permanent_loss = []  # Track cards lost permanently
        self.gold = 0  # Track the player's gold

    def draw(self, deck):
        card = deck.draw_card()
        if card:
            self.hand.append(card)
        return card

    def play_card(self):
        return self.hand.pop(0) if self.hand else None

    def add_cards(self, cards):
        self.hand.extend(cards)

    def remove_random_cards(self, num_cards):
        lost_cards = []
        for _ in range(num_cards):
            if self.hand:
                card = self.hand.pop(random.randint(0, len(self.hand) - 1))  # Randomly remove a card
                lost_cards.append(card)
        self.permanent_loss.extend(lost_cards)  # Store lost cards permanently
        print(f"{self.name} lost these cards permanently: {', '.join(str(card) for card in lost_cards)}")

    def show_hand(self):
        return ', '.join(str(card) for card in self.hand)

    def __str__(self):
        return f"{self.name} has {len(self.hand)} cards: {', '.join(str(card) for card in self.hand[:5])} {'...' if len(self.hand) > 5 else ''}"

    def earn_gold(self, amount):
        self.gold += amount
        print(f"{self.name} earned {amount} gold!")

    def can_afford(self, cost):
        return self.gold >= cost

    def buy_card(self, card, cost):
        if self.can_afford(cost):
            self.gold -= cost
            self.add_cards([card])
            print(f"{self.name} bought {card} for {cost} gold.")
        else:
            print(f"{self.name} doesn't have enough gold to buy {card}.")

class WarGame:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        self.player1 = Player("User")
        self.player2 = self.generate_new_computer()  # Generate a new computer opponent
        self.deal_cards()
        self.locations = ["Forest", "Desert", "Mountain", "Swamp", "Ocean", "Capital City"]
        self.current_location = 0  # Starting location for player
        self.max_rounds = 10  # Set a limit on the number of rounds for a quicker game
        self.round_count = 0  # To track the number of rounds played
        self.game_over = False  # To control game flow and exit condition

    def generate_new_computer(self):
        computer = Player("Computer")
        card_count = random.randint(5, 15)
        for _ in range(card_count):
            computer.draw(self.deck)
        return computer

    def deal_cards(self):
        # Random card count for each player (between 5 and 15 cards)
        player1_card_count = random.randint(5, 15)
        player2_card_count = random.randint(5, 15)

        # Clear hands before re-dealing to avoid duplication
        self.player1.hand = []
        self.player2.hand = []

        for _ in range(player1_card_count):
            self.player1.draw(self.deck)

        for _ in range(player2_card_count):
            self.player2.draw(self.deck)

        print(f"Starting with {len(self.player1.hand)} cards for {self.player1.name} and {len(self.player2.hand)} cards for {self.player2.name}")

    def calculate_units(self, card, environment):
        base_units = card.value * random.randint(1, 3)  # Random factor for small variability
        environment_multiplier = {
            "Forest": 1.0, "Desert": 1.1, "Mountain": 1.2, "Swamp": 1.05, "Ocean": 1.3, "Capital City": 1.5
        }
        
        fluctuation = random.uniform(0.8, 1.2)  # 80% to 120% fluctuation
        return int(base_units * environment_multiplier[environment] * fluctuation)

    def play_round(self):
        if not self.player1.hand or not self.player2.hand:
            return False

        input(f"Round {self.round_count + 1} in {self.locations[self.current_location]}: Press Enter to play your card...")
        print(f"Available cards in your hand: {self.player1.show_hand()}")
        card1 = self.player1.play_card()
        card2 = random.choice(self.player2.hand)  # Randomized card for computer

        # Choose random environment for this round
        environment = self.locations[self.current_location]
        units1 = self.calculate_units(card1, environment)
        units2 = self.calculate_units(card2, environment)

        print(f"{self.player1.name} plays {card1} with {units1} units")
        print(f"{self.player2.name} plays {card2} with {units2} units")

        # Determine round winner and card assignment
        gold_won = random.randint(10, 50)  # Random gold awarded for winning the round
        if units1 > units2:
            print(f"{self.player1.name} wins the round with {units1 - units2} units remaining!")
            self.player1.add_cards([card1, card2])
            self.player1.earn_gold(gold_won)
            # Loses random cards (from losing player)
            num_cards_lost = random.randint(1, 3)
            print(f"{self.player2.name} loses {num_cards_lost} cards permanently!")
            self.player2.remove_random_cards(num_cards_lost)
        elif units2 > units1:
            print(f"{self.player2.name} wins the round with {units2 - units1} units remaining!")
            self.player2.add_cards([card1, card2])
            self.player2.earn_gold(gold_won)
            # Loses random cards (from losing player)
            num_cards_lost = random.randint(1, 3)
            print(f"{self.player1.name} loses {num_cards_lost} cards permanently!")
            self.player1.remove_random_cards(num_cards_lost)
        else:
            print("It's a tie! Both sides lose their units.")
            # Both players lose random cards permanently
            num_cards_lost = random.randint(1, 3)
            print(f"Both players lose {num_cards_lost} cards permanently!")
            self.player1.remove_random_cards(num_cards_lost)
            self.player2.remove_random_cards(num_cards_lost)

        print(f"\nCurrent card count: {self.player1.name}: {len(self.player1.hand)} cards | {self.player2.name}: {len(self.player2.hand)} cards\n")
        print(f"{self.player1.name} has {self.player1.gold} gold | {self.player2.name} has {self.player2.gold} gold\n")
        
        # If the computer has no cards left, replace it
        if not self.player2.hand:
            print(f"{self.player2.name} has been eliminated. A new computer opponent is appearing!")
            self.player2 = self.generate_new_computer()

        return True

    def trade_shop(self, player):
        print("\nWelcome to the Trade Shop!")
        available_cards = [
            ("2 of Hearts", 20), ("Ace of Diamonds", 50), ("Wizard of Spades", 100),
            ("Dragon of Clubs", 150), ("Phoenix of Moons", 200)
        ]
        
        for idx, (card_name, price) in enumerate(available_cards, 1):
            print(f"{idx}. {card_name} - {price} gold")
        
        while True:
            choice = input(f"Choose a card to buy (1-{len(available_cards)}) or 0 to exit: ")
            if choice == "0":
                break
            elif choice.isdigit() and 1 <= int(choice) <= len(available_cards):
                card_name, price = available_cards[int(choice) - 1]
                # Buy the card if player has enough gold
                player.buy_card(Card("Random", card_name, random.randint(10, 200)), price)
            else:
                print("Invalid choice. Please try again.")
            
            break

    def ask_continue_or_trade(self):
        print("\nOptions:")
        print("1. Continue to next round")
        print("2. Trade cards in shop")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            self.current_location = (self.current_location + 1) % len(self.locations)
            self.round_count += 1
            if self.round_count >= self.max_rounds:
                print("Game over. Maximum rounds reached.")
                self.game_over = True
        elif choice == "2":
            self.trade_shop(self.player1)
        elif choice == "3":
            print("Exiting the game.")
            self.game_over = True
        else:
            print("Invalid choice. Please try again.")
    
    def play_game(self):
        print("Starting the game of War!")
        while not self.game_over:
            if not self.play_round():
                break
            self.ask_continue_or_trade()
        
        print("Game over!")
        if self.player1.gold > 0:
            print(f"{self.player1.name} had {self.player1.gold} gold at the end.")

if __name__ == "__main__":
    while True:
        print("\nWelcome to the War Card Game!")
        print("1. Start a new game")
        print("2. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            game = WarGame()
            game.play_game()
        elif choice == "2":
            print("Thank you for playing! Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")




Welcome to the War Card Game!
1. Start a new game
2. Exit
Starting with 9 cards for User and 15 cards for Computer
Starting the game of War!
Available cards in your hand: 2 of Hearts, 4 of Diamonds, King of Hearts, 6 of Clubs, 8 of Spades, Wizard of Diamonds, 9 of Diamonds, 8 of Moons, 4 of Clubs
User plays 2 of Hearts with 4 units
Computer plays Phoenix of Stars with 34 units
Computer wins the round with 30 units remaining!
Computer earned 38 gold!
User loses 3 cards permanently!
User lost these cards permanently: 8 of Moons, Wizard of Diamonds, 4 of Diamonds

Current card count: User: 5 cards | Computer: 17 cards

User has 0 gold | Computer has 38 gold


Options:
1. Continue to next round
2. Trade cards in shop
3. Exit
Available cards in your hand: King of Hearts, 6 of Clubs, 8 of Spades, 9 of Diamonds, 4 of Clubs
User plays King of Hearts with 49 units
Computer plays 7 of Spades with 19 units
User wins the round with 30 units remaining!
User earned 21 gold!
Computer loses 2 cards p